# Análise Quantitativa do Trade-off entre Especialização e Generalização em LLMs via Fine-Tuning

**Disciplina:** TÓPICOS ESPECIAIS EM BANCO DE DADOS - NLP  
**Trabalho Prático:** Análise Quantitativa do Trade-off entre Especialização e Generalização em LLMs via Fine-Tuning

- Allan Carvalho de Aguiar (22153696)
- Beatriz Emily Silva Aguiar (22154303)
- Juíle Yoshie Sarkis Hanada (22251135)

**Professor:** Altigran Soares e André Carvalho  
**Data de Entrega:** 23/06/2025    

---


In [ ]:
!pip install -U datasets
!pip install transformers
!pip install torch
!pip install accelerate
!pip install bitsandbytes
!pip install huggingface-hub
!pip install -U peft
!pip install -U trl
!pip install -U deepeval
!pip install -U pytest

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [ ]:
import os

if not os.path.exists("spider_data"):
  !gdown "1403EGqzIDoHMdQF4c9Bkyl7dZLZ5Wt6J"
  !unzip -q spider_data.zip
  !rm spider_data.zip
else:
  pass

Downloading...
From (original): https://drive.google.com/uc?id=1403EGqzIDoHMdQF4c9Bkyl7dZLZ5Wt6J
From (redirected): https://drive.google.com/uc?id=1403EGqzIDoHMdQF4c9Bkyl7dZLZ5Wt6J&confirm=t&uuid=ad6e112c-d23a-43a9-be1b-bb64616b8004
To: /content/spider_data.zip
100% 206M/206M [00:02<00:00, 91.0MB/s]


In [ ]:
from google.colab import userdata
from huggingface_hub import login

hf_token = userdata.get('HF_TOKEN')

login(token=hf_token)

In [ ]:
from datasets import load_dataset
spider_dataset = load_dataset("spider")
spider_dataset

README.md:   0%|          | 0.00/5.51k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/831k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/126k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1034 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
        num_rows: 7000
    })
    validation: Dataset({
        features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
        num_rows: 1034
    })
})

### Dataset de Fine-Tuning e Dataset de Avaliação de Tarefa

In [ ]:
train_spider = spider_dataset['train']
validation_spider = spider_dataset['validation']

del spider_dataset

### Dataset de Avaliação de Generalização

In [ ]:
mmlu_dataset = load_dataset("cais/mmlu", "all")
mmlu_dataset

README.md:   0%|          | 0.00/53.2k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/138k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/3.50M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/408k [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/76.5k [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/14042 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1531 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/285 [00:00<?, ? examples/s]

Generating auxiliary_train split:   0%|          | 0/99842 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 14042
    })
    validation: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 1531
    })
    dev: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 285
    })
    auxiliary_train: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 99842
    })
})

### Criação da Suíte de Avaliação Composta

In [ ]:
from datasets import concatenate_datasets, DatasetDict
import pandas as pd

# 1. Definir as subcategorias escolhidas e o número de questões
NUM_QUESTIONS_PER_CATEGORY = 50
CATEGORIES = {
    "stem": "college_computer_science", # Changed 'computer_science' to a valid subject name
    "humanities": "philosophy",
    "social_sciences": "econometrics" # Changed from 'professional_economics' to 'econometrics'
}

# Lista para armazenar os datasets de cada categoria
selected_datasets = []

print("Iniciando o processo de criação da suíte de avaliação MMLU...")
print("-" * 50)

# 2. Loop para carregar, embaralhar e selecionar dados de cada categoria
for category, mmlu_subject in CATEGORIES.items():
    print(f"Processando categoria: {category.upper()} (Subcategoria MMLU: {mmlu_subject})")

    # Carrega a subcategoria específica do MMLU
    current_subject_dataset_dict = load_dataset("cais/mmlu", mmlu_subject)

    # Usamos o split 'test' que é padrão para avaliação
    dataset = current_subject_dataset_dict['test']

    # Embaralha o dataset para garantir uma seleção aleatória e reprodutível (com seed)
    # e seleciona o número desejado de questões
    shuffled_dataset = dataset.shuffle(seed=42).select(range(NUM_QUESTIONS_PER_CATEGORY))

    selected_datasets.append(shuffled_dataset)
    print(f"-> {len(shuffled_dataset)} questões selecionadas para '{mmlu_subject}'.")

print("-" * 50)

# 3. Combinar os três datasets em um único dataset final
final_evaluation_dataset = concatenate_datasets(selected_datasets)

del selected_datasets

# Embaralhar o dataset final para misturar as categorias
final_evaluation_dataset = final_evaluation_dataset.shuffle(seed=42)

# 4. Criar o DatasetDict final, como solicitado
evaluation_suite = DatasetDict({
    "evaluation": final_evaluation_dataset
})

print("Suíte de avaliação criada com sucesso!")
print("\nEstrutura final do DatasetDict:")
print(evaluation_suite)

# 5. Verificação opcional: contar as questões por disciplina para garantir a divisão correta
print("\nVerificando a distribuição das questões por disciplina:")
df = evaluation_suite["evaluation"].to_pandas()
subject_counts = df['subject'].value_counts()
print(subject_counts)

Iniciando o processo de criação da suíte de avaliação MMLU...
--------------------------------------------------
Processando categoria: STEM (Subcategoria MMLU: college_computer_science)


test-00000-of-00001.parquet:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/6.25k [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/6.81k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

...

dev-00000-of-00001.parquet:   0%|          | 0.00/4.54k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/114 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/12 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

-> 50 questões selecionadas para 'econometrics'.
--------------------------------------------------
Suíte de avaliação criada com sucesso!

Estrutura final do DatasetDict:
DatasetDict({
    evaluation: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 150
    })
})

Verificando a distribuição das questões por disciplina:
subject
philosophy                  50
econometrics                50
college_computer_science    50
Name: count, dtype: int64


In [ ]:
def criar_prompt_llama3(schema, question):
    """
    Cria um prompt de few-shot no formato específico e recomendado
    para a família Llama 3 Instruct.
    """
    # Instrução principal movida para o 'role' de sistema.
    system_prompt = "Você é um especialista em SQL. Sua tarefa é gerar uma consulta SQL precisa a partir de uma pergunta em linguagem natural e o esquema do banco de dados fornecido. Não adicione nenhuma explicação, apenas o código SQL."

    # A estrutura f-string principal que formata a conversa.
    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>
### Exemplo 1 (Fácil)
### Esquema do Banco de Dados:
# CREATE TABLE cars_data (cylinders INTEGER)
### Pergunta:
# What is the number of cars with more than 4 cylinders?
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
SELECT COUNT(*) FROM cars_data WHERE cylinders > 4<|eot_id|><|start_header_id|>user<|end_header_id|>
### Exemplo 2 (Médio)
### Esquema do Banco de Dados:
# CREATE TABLE concert (stadium_id INTEGER)
# CREATE TABLE stadium (stadium_id INTEGER, name TEXT)
### Pergunta:
# For each stadium, how many concerts are there?
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
SELECT T2.name, COUNT(*) FROM concert AS T1 JOIN stadium AS T2 ON T1.stadium_id = T2.stadium_id GROUP BY T2.name<|eot_id|><|start_header_id|>user<|end_header_id|>
### Exemplo 3 (Difícil)
### Esquema do Banco de Dados:
# CREATE TABLE continents (cont_id INTEGER, continent TEXT)
# CREATE TABLE countries (country_id INTEGER, country_name TEXT, continent INTEGER)
# CREATE TABLE car_makers (maker_id INTEGER, maker TEXT, country INTEGER)
### Pergunta:
# Which countries in Europe have at least 3 car manufacturers?
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
SELECT T1.country_name FROM countries AS T1 JOIN continents AS T2 ON T1.continent = T2.cont_id JOIN car_makers AS T3 ON T1.country_id = T3.country WHERE T2.continent = 'Europe' GROUP BY T1.country_name HAVING COUNT(*) >= 3<|eot_id|><|start_header_id|>user<|end_header_id|>
### Exemplo 4 (Extra Difícil)
### Esquema do Banco de Dados:
# CREATE TABLE country (Code TEXT, Name TEXT, LifeExpectancy REAL)
# CREATE TABLE country_language (CountryCode TEXT, Language TEXT, IsOfficial TEXT)
### Pergunta:
# What is the average life expectancy in the countries where English is not the official language?
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
SELECT AVG(LifeExpectancy) FROM country WHERE Name NOT IN (SELECT T1.Name FROM country AS T1 JOIN country_language AS T2 ON T1.Code = T2.CountryCode WHERE T2.Language = "English" AND T2.IsOfficial = "T")<|eot_id|><|start_header_id|>user<|end_header_id|>
### Agora é a sua vez.

### Esquema do Banco de Dados:
# {schema}
### Pergunta:
# {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
    return prompt

def criar_prompt_mistral(schema, question):
    """
    Cria um prompt de few-shot no formato específico para o Mistral-7B-Instruct-v0.2.
    """
    # O prompt para o Mistral usa as tags [INST] e [/INST] para delimitar as instruções do usuário.
    # A resposta do assistente vem logo após a tag de fechamento [/INST].
    prompt = f"""<s>[INST]Você é um especialista em SQL. Sua tarefa é gerar uma consulta SQL precisa a partir de uma pergunta em linguagem natural e o esquema do banco de dados fornecido. Não adicione nenhuma explicação, apenas o código SQL.

### Exemplo 1
### Esquema do Banco de Dados:
# CREATE TABLE head (age INTEGER, name TEXT)
# CREATE TABLE department (name TEXT, num_employees INTEGER)
### Pergunta:
# List all department names that have more than 20 employees.[/INST]
SELECT name FROM department WHERE num_employees > 20</s>
[INST]### Exemplo 2
### Esquema do Banco de Dados:
# CREATE TABLE singer (Name TEXT, Country TEXT, Age INTEGER)
# CREATE TABLE song (Title TEXT, Singer TEXT)
### Pergunta:
# What are the names of singers from France?[/INST]
SELECT Name FROM singer WHERE Country = 'France'</s>
[INST]### Exemplo 3
### Esquema do Banco de Dados:
# CREATE TABLE concert (Theme TEXT, Year INTEGER)
# CREATE TABLE singer_in_concert (concert_Theme TEXT, singer_Name TEXT)
### Pergunta:
# Count the number of singers in concerts for each theme.[/INST]
SELECT T1.Theme, count(*) FROM concert AS T1 JOIN singer_in_concert AS T2 ON T1.Theme  =  T2.concert_Theme GROUP BY T1.Theme</s>
[INST]### Fim dos Exemplos. Agora, sua vez.

### Esquema do Banco de Dados:
# {schema}
### Pergunta:
# {question}[/INST]
"""
    return prompt

In [ ]:
schema_teste = "CREATE TABLE students (name TEXT, grade REAL)"
pergunta_teste = "What are the names of students with a grade above 8.5?"

prompt_final = criar_prompt_llama3(schema_teste, pergunta_teste)
print(prompt_final)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Você é um especialista em SQL. Sua tarefa é gerar uma consulta SQL precisa a partir de uma pergunta em linguagem natural e o esquema do banco de dados fornecido. Não adicione nenhuma explicação, apenas o código SQL.<|eot_id|><|start_header_id|>user<|end_header_id|>
### Exemplo 1 (Fácil)
### Esquema do Banco de Dados:
# CREATE TABLE cars_data (cylinders INTEGER)
### Pergunta:
# What is the number of cars with more than 4 cylinders?
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
SELECT COUNT(*) FROM cars_data WHERE cylinders > 4<|eot_id|><|start_header_id|>user<|end_header_id|>
### Exemplo 2 (Médio)
### Esquema do Banco de Dados:
# CREATE TABLE concert (stadium_id INTEGER)
# CREATE TABLE stadium (stadium_id INTEGER, name TEXT)
### Pergunta:
# For each stadium, how many concerts are there?
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
SELECT T2.name, COUNT(*) FROM concert AS T1 JOIN stadium AS T2 ON T1.stadium_

In [ ]:
import json

with open('spider_data/tables.json', 'r') as f:
    schemas_data = json.load(f)

# Criar um dicionário para acesso rápido aos esquemas usando o db_id como chave
schemas_map = {db['db_id']: db for db in schemas_data}

# --- Vamos pegar um exemplo e encontrar seu esquema ---
exemplo_pergunta = validation_spider[0]
db_id_do_exemplo = exemplo_pergunta['db_id']

print(f"Pergunta de Exemplo: '{exemplo_pergunta['question']}'")
print(f"ID do Banco de Dados: {db_id_do_exemplo}")

# Agora, use o db_id para buscar o esquema no nosso mapa
esquema_correspondente = schemas_map[db_id_do_exemplo]

print("\n--- Esquema Encontrado no 'tables.json' ---")
print(f"Nomes das Tabelas: {esquema_correspondente['table_names_original']}")

print("\nNomes das Colunas (índice da tabela, nome da coluna):")
# A coluna é uma lista de listas: [índice da tabela, 'nome_da_coluna']
for col in esquema_correspondente['column_names_original']:
    print(f"  Tabela '{esquema_correspondente['table_names_original'][col[0]]}': {col[1]}")

Pergunta de Exemplo: 'How many singers do we have?'
ID do Banco de Dados: concert_singer

--- Esquema Encontrado no 'tables.json' ---
Nomes das Tabelas: ['stadium', 'singer', 'concert', 'singer_in_concert']

Nomes das Colunas (índice da tabela, nome da coluna):
  Tabela 'singer_in_concert': *
  Tabela 'stadium': Stadium_ID
  Tabela 'stadium': Location
  Tabela 'stadium': Name
  Tabela 'stadium': Capacity
  Tabela 'stadium': Highest
  Tabela 'stadium': Lowest
  Tabela 'stadium': Average
  Tabela 'singer': Singer_ID
  Tabela 'singer': Name
  Tabela 'singer': Country
  Tabela 'singer': Song_Name
  Tabela 'singer': Song_release_year
  Tabela 'singer': Age
  Tabela 'singer': Is_male
  Tabela 'concert': concert_ID
  Tabela 'concert': concert_Name
  Tabela 'concert': Theme
  Tabela 'concert': Stadium_ID
  Tabela 'concert': Year
  Tabela 'singer_in_concert': concert_ID
  Tabela 'singer_in_concert': Singer_ID


## Carregar o modelo llama 3

>  é o modelo base usado para aprender a responder perguntas em linguagem natural com comandos SQL.
> Dada uma pergunta em linguagem natural e um esquema de banco de dados, o modelo precisa gerar a query SQL correta.

>Esquema: CREATE TABLE alunos (nome TEXT, idade INTEGER)

>Pergunta: Quantos alunos têm mais de 18 anos?

>SELECT COUNT(*) FROM alunos WHERE idade > 18

| Termo         | O que é                                                                                                                                                              |
| ------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `MODEL_ID`    | Nome do modelo no [Hugging Face Hub](https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct). Aqui, o LLaMA 3.2 com 3B parâmetros, ajustado para seguir instruções. |
| `COMMIT_HASH` | Um **ID único de versão** do modelo. Garante que você sempre carregue a **mesma versão** (checkpoint) para reprodutibilidade.                                        |

| Parte             | Explicação                                                                                                                  |
| ----------------- | --------------------------------------------------------------------------------------------------------------------------- |
| `tokenizer`       | É o componente que transforma **texto em números** (tokens) para o modelo entender. Ex: "Oi, tudo bem?" → `[342, 29, 812]`. |
| `from_pretrained` | Carrega o tokenizador padrão usado durante o pré-treinamento do modelo.                                                     |
| `token=hf_token`  | Seu token pessoal da Hugging Face (para acessar modelos privados/gated).                                                    |

| Config                     | Explicação                                                                                                                                                  |
| -------------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `load_in_4bit=True`        | Carrega o modelo em **4 bits** (em vez de 16/32), economizando muita memória de GPU. Ideal para rodar modelos grandes em máquinas limitadas (como o Colab). |
| Outras opções (comentadas) | Servem para **refinar o tipo de quantização**, mas não são obrigatórias para funcionar.                                                                     |

| Parâmetro                        | O que faz                                                                   |
| -------------------------------- | --------------------------------------------------------------------------- |
| `from_pretrained(...)`           | Carrega os pesos do modelo base LLaMA 3 (na versão exata do `COMMIT_HASH`). |
| `quantization_config=bnb_config` | Diz para carregar em 4-bit, via configuração anterior.                      |
| `device_map="auto"`              | Deixa o sistema distribuir o modelo entre CPU/GPU automaticamente.          |
| `torch_dtype=torch.bfloat16`     | Usa o tipo `bfloat16`, que é mais leve que float32 (recomendado para LLMs). |



In [ ]:
import requests
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from tqdm import tqdm
import time

# --- Configuração ---
MODEL_ID = "meta-llama/Llama-3.2-3B-Instruct"

COMMIT_HASH = "0cb88a4f764b7a12671c53f0838cd831a0843b95"

print("Carregando o tokenizador...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, COMMIT_HASH, token=hf_token)

# Configuração de quantização para carregar o modelo em 4-bit
bnb_config = BitsAndBytesConfig(
    # load_in_8bit=True,
    load_in_4bit=True,
    # bnb_4bit_quant_type="nf4",
    # bnb_4bit_compute_dtype=torch.bfloat16,
    # bnb_4bit_use_double_quant=True,
)

# Carregar o modelo base com quantização
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    revision=COMMIT_HASH,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM

# # Modelo leve e público, ideal para testar pipeline
# MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# print("Carregando o tokenizador...")
# tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

# print("Carregando o modelo...")
# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_ID,
#     device_map="auto" if torch.cuda.is_available() else None,
# )



Carregando o tokenizador...


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

# 🧩 Explicação da função `formatar_schema`

A função `formatar_schema` é responsável por **converter a estrutura de um banco de dados (esquema)** presente no dataset **Spider** em formato textual, usando instruções do tipo `CREATE TABLE`.

Ela é útil porque modelos de linguagem como o LLaMA precisam **entender o esquema das tabelas** (nomes, colunas e tipos) em formato natural — como se fosse um código SQL real — para conseguir responder corretamente a perguntas como:

- *"Quais carros custam mais de 10 mil?"*
- *"Quantos concertos existem por estádio?"*

---

## ✅ O que ela faz?

- Recebe um exemplo do Spider (um dicionário com uma pergunta e o `db_id` do banco de dados).
- Busca no `schemas_map` (um dicionário carregado antes) o esquema completo do banco com aquele `db_id`.
- Gera uma string no formato:

```sql
CREATE TABLE nome_tabela (
    nome_coluna tipo_coluna,
    ...
)
```

- Repete isso para todas as tabelas do banco e junta tudo em uma string final.

---

## 📥 Parâmetros

- `exemplo_spider`: Um `dict` representando um exemplo do dataset Spider. Deve conter a chave `db_id`.

---

## 📤 Retorno

- Uma `string` com a estrutura completa do banco em formato SQL (`CREATE TABLE`).
- Caso o `db_id` não esteja em `schemas_map`, retorna uma mensagem de erro.

---


---

## 🧪 Exemplo de uso

### 🟦 Entrada (exemplo de `exemplo_spider`)

```python
exemplo_spider = {
    "db_id": "concert_singer"
}
```

Suponha que o `schemas_map["concert_singer"]` contenha:

```python
{
    "table_names_original": ["singer", "concert"],
    "column_names_original": [
        (0, "singer_id"), (0, "name"),
        (1, "concert_id"), (1, "theme")
    ],
    "column_types": [
        "int", "text",
        "int", "text"
    ]
}
```

---

### 🟩 Saída da função `formatar_schema(exemplo_spider)`

```sql
CREATE TABLE singer (singer_id int, name text)
CREATE TABLE concert (concert_id int, theme text)
```

---

Esse formato de saída é usado diretamente na construção de prompts para modelos LLMs como LLaMA, permitindo que eles entendam como o banco de dados está estruturado para gerar queries SQL corretas com base em perguntas em linguagem natural.


In [ ]:
def formatar_schema(exemplo_spider):
    """
    Formata o esquema de um banco de dados em formato CREATE TABLE,
    utilizando um dicionário de esquemas pré-carregado (schemas_map).

    Args:
        exemplo_spider (dict): Um único exemplo do dataset Spider.

    Returns:
        str: Uma string com as declarações CREATE TABLE para o banco de dados.
    """
    db_id = exemplo_spider['db_id']

    if db_id not in schemas_map:
        return f"ERRO: Esquema para o db_id '{db_id}' não encontrado."

    schema_info = schemas_map[db_id]

    # Extrair as listas relevantes do esquema
    table_names = schema_info['table_names_original']
    column_names = schema_info['column_names_original']
    column_types = schema_info['column_types']

    schema_parts = []
    # Iterar sobre cada tabela para construir sua declaração
    for table_idx, table_name in enumerate(table_names):
        cols_da_tabela = []

        # Iterar sobre o ÍNDICE das colunas para acessar as listas paralelas
        for i in range(len(column_names)):
            # Verificar se a coluna atual (índice i) pertence à tabela atual (table_idx)
            if column_names[i][0] == table_idx:
                col_name = column_names[i][1]
                col_type = column_types[i]
                cols_da_tabela.append(f"{col_name} {col_type}")

        schema_parts.append(f"CREATE TABLE {table_name} ({', '.join(cols_da_tabela)})")

    return "\n".join(schema_parts)

---

## 🧪 Execução da Avaliação com Llama 3

Esse bloco de código realiza a **inferência (geração de texto)** em cima do conjunto de validação do dataset Spider, utilizando um modelo LLaMA 3 fine-tunado com LoRA. A avaliação segue os seguintes passos:

---

### 🔄 Etapas do processo

1. **Desativar o cálculo de gradientes com `torch.no_grad()`**  
   Isso economiza memória e acelera o processo, já que não estamos treinando o modelo, apenas gerando saídas.

2. **Iterar sobre exemplos da base de validação**  
   É utilizado `validation_spider.select(range(50))` para limitar a avaliação aos primeiros 50 exemplos.

3. **Para cada exemplo:**
   - Extrai-se a **pergunta** (`question`), a **query correta** (`query`) e o identificador do banco de dados (`db_id`);
   - Formata-se o **esquema SQL** com a função `formatar_schema`;
   - Cria-se o **prompt completo** com `criar_prompt_llama3`, unindo o esquema e a pergunta;
   - Tokeniza-se esse prompt com `tokenizer(..., return_tensors="pt")`;
   - Usa-se o modelo (`model.generate`) para gerar uma resposta (a query SQL);
   - A saída é **decodificada** para string com `tokenizer.decode`;
   - A resposta gerada é armazenada junto com o `db_id`, `pergunta`, `query_correta`, `query_gerada` e caminho do banco (`db_path`).

4. **Ao final**:
   - Os resultados são salvos em um DataFrame `df_resultados`;
   - Esse DataFrame é exportado para CSV: `"llama3_local_baseline_resultados.csv"`.

---

### 📦 Exemplo de entrada (no loop)

```python
{
  "db_id": "concert_singer",
  "question": "How many singers are there?",
  "query": "SELECT COUNT(*) FROM singer"
}
```

### 📤 Exemplo de saída esperada no CSV

| db_id         | pergunta              | query_correta                  | query_gerada                  | db_path                                               |
|---------------|------------------------|--------------------------------|-------------------------------|--------------------------------------------------------|
| concert_singer | How many singers are there? | SELECT COUNT(*) FROM singer | SELECT COUNT(*) FROM singer | spider_data/database/concert_singer/concert_singer.sqlite |

---

✅ **Observação importante**: o código pode ser adaptado para qualquer modelo que aceite prompts no formato da função `criar_prompt_llama3`.



In [ ]:
# --- Execução da Avaliação ---
resultados = []
DB_DIR = "spider_data/database"
# `torch.no_grad()` desativa o cálculo de gradientes, economizando memória e acelerando a inferência.
with torch.no_grad():
    for i, exemplo in enumerate(tqdm(validation_spider.select(range(50)), desc="Avaliando Llama 3")):
        pergunta = exemplo['question']
        query_correta = exemplo['query']
        db_id = exemplo['db_id']

        schema_formatado = formatar_schema(exemplo)
        prompt_completo = criar_prompt_llama3(schema_formatado, pergunta)
        path_db = os.path.join(DB_DIR, db_id, f"{db_id}.sqlite")

        # 1. Tokenizar o prompt
        # O prompt é convertido em números que o modelo entende.
        inputs = tokenizer(prompt_completo, return_tensors="pt").to(model.device)

        # 2. Gerar a resposta com o modelo
        # O modelo gera a continuação do prompt tokenizado.
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            eos_token_id=tokenizer.eos_token_id,
            do_sample=False # Usamos do_sample=False para obter resultados mais consistentes
        )

        # 3. Decodificar a resposta
        # Os números gerados são convertidos de volta para texto.
        # `outputs[0]` pega o primeiro (e único) resultado do lote.
        # `inputs.input_ids.shape[1]:` fatia a saída para remover o prompt original e manter apenas o texto gerado.
        query_gerada = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True).strip()

        resultados.append({
            "db_id": exemplo['db_id'],
            "pergunta": pergunta,
            "query_correta": query_correta,
            "query_gerada": query_gerada,
            "db_path": path_db,
        })

        # Opcional: Limpar o cache da GPU em loops longos
        # torch.cuda.empty_cache()

df_resultados = pd.DataFrame(resultados)
df_resultados.to_csv("llama3_local_baseline_resultados.csv", index=False)

print("\nAvaliação Local com Llama 3 concluída! Resultados salvos em 'llama3_local_baseline_resultados.csv'")
# print(df_resultados.head())

Avaliando Llama 3:   0%|          | 0/50 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Avaliando Llama 3:   2%|▏         | 1/50 [00:01<01:00,  1.23s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Avaliando Llama 3:   4%|▍         | 2/50 [00:01<00:34,  1.38it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Avaliando Llama 3:   6%|▌         | 3/50 [00:02<00:46,  1.01it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRAN


Avaliação Local com Llama 3 concluída! Resultados salvos em 'llama3_local_baseline_resultados.csv'


---

## 🧪 Função `verificar_execucao`: Validação por Execução SQL

Esta função é usada para **verificar se a query SQL gerada pelo modelo retorna o mesmo resultado que a query correta (padrão-ouro)** em um banco de dados real do Spider.

---

### 📌 Objetivo

Comparar **execução real** da query gerada versus a query correta no mesmo banco, e retornar:

- `True` → se os dois resultados forem **iguais**
- `False` → se a query falhar ou retornar valores diferentes

---

### 🧠 Lógica da Função

```python
def verificar_execucao(query_gerada, query_correta, db_path):
```

#### 🔍 Etapas:

1. **Verificação inicial**:
   - Se a query gerada estiver vazia ou não for string → retorna `False`.

2. **Conexão com o banco de dados**:
   - Usa `sqlite3.connect(...)` em modo somente leitura (`mode=ro`) no banco SQLite associado ao exemplo.

3. **Execução das queries**:
   - Executa a `query_correta` e armazena o resultado.
   - Executa a `query_gerada` pelo modelo e armazena o resultado.

4. **Normalização dos resultados**:
   - Converte todos os elementos para `string` para evitar erros ao ordenar/comparar (`TypeError`).
   - Usa `sorted(...)` para garantir que a ordem das linhas não afete a comparação (já que SQL não ordena por padrão sem `ORDER BY`).

5. **Comparação**:
   - Retorna `True` se os resultados ordenados forem idênticos.

6. **Tratamento de erros**:
   - Qualquer exceção (por exemplo, erro de sintaxe SQL) → retorna `False`.

7. **Fechamento seguro da conexão**:
   - Garante o `conexao.close()` sempre, mesmo em caso de erro.

---

### 📦 Exemplo de uso

```python
verificar_execucao(
  "SELECT COUNT(*) FROM singer",
  "SELECT COUNT(*) FROM singer",
  "spider_data/database/concert_singer/concert_singer.sqlite"
)
# Saída: True
```

---

✅ **Resumo**: Essa função valida **corretude semântica** da query gerada, executando no banco real. É uma métrica mais robusta do que simplesmente comparar textos.


In [ ]:
import sqlite3

# O nome do arquivo gerado pelo script de execução local
ARQUIVO_DE_RESULTADOS = "llama3_local_baseline_resultados.csv"

def verificar_execucao(query_gerada, query_correta, db_path):
    """
    Verifica se a query gerada produz o mesmo resultado que a query correta.
    Retorna True para sucesso, False para falha.
    """
    # Se o modelo não gerou nada ou retornou um erro, é uma falha.
    if not isinstance(query_gerada, str) or not query_gerada.strip():
        return False

    conexao = None
    try:
        # Conectar ao banco de dados específico deste exemplo
        conexao = sqlite3.connect(f"file:{db_path}?mode=ro", uri=True) # Modo somente leitura
        cursor = conexao.cursor()

        # Executar la query correta (padrão ouro)
        cursor.execute(query_correta)
        resultado_correto = cursor.fetchall()

        # Tentar executar la query gerada pelo modelo
        cursor.execute(query_gerada)
        resultado_gerado = cursor.fetchall()

        # Converter todos os elementos dos resultados para string antes de ordenar para evitar TypeError
        # Isso garante que todos os elementos sejam comparáveis.
        resultado_gerado_str = [tuple(str(item) for item in row) for row in resultado_gerado]
        resultado_correto_str = [tuple(str(item) for item in row) for row in resultado_correto]

        # print(resultado_gerado_str)
        # print(resultado_correto_str)

        # A ordem dos resultados de uma consulta SQL não é garantida sem ORDER BY.
        # Portanto, a comparação mais robusta é ordenar os resultados antes de comparar.
        return sorted(resultado_gerado_str) == sorted(resultado_correto_str)

    except sqlite3.Error:
        # Qualquer erro de sintaxe ou execução na query gerada é uma falha.
        return False
    finally:
        # Garantir que a conexão seja sempre fechada.
        if conexao:
            conexao.close()

---

## 📊 Verificação por Execução e Relatório Final

Este trecho de código carrega os resultados gerados pelo modelo (em CSV), verifica se cada query gerada está **corretamente executável** e retorna **um relatório de acurácia de execução**.

---

### 📁 Leitura dos Resultados

```python
df_resultados = pd.read_csv(ARQUIVO_DE_RESULTADOS)
```

- Tenta carregar o arquivo `llama3_local_baseline_resultados.csv` com as queries geradas.
- Se o arquivo não existir, imprime uma mensagem de erro e encerra a execução (`exit()`).

---

### 🔍 Verificação de Execução

```python
for index, row in df_resultados.iterrows():
```

- Para cada linha (exemplo) do DataFrame:
  - Executa `verificar_execucao(...)` passando a query gerada, a correta e o caminho para o banco.
  - Se os resultados forem equivalentes, conta como **sucesso**.
  - Armazena o status (`Sucesso` ou `Falha`) numa nova lista `resultados_detalhados`.

---

### 📋 Registro dos Resultados

```python
df_resultados['status_execucao'] = resultados_detalhados
df_resultados.to_csv("llama3_local_resultados_verificados.csv")
```

- Cria uma nova coluna `status_execucao` no DataFrame com o status de cada verificação.
- Salva o CSV atualizado com os resultados verificados.

---

### 📈 Relatório Final

```python
print("--- Relatório de Baseline (Execução Local) ---")
```

- Exibe:
  - Total de amostras avaliadas
  - Quantas foram sucesso ou falha
  - **Taxa de Sucesso** (% de queries geradas que retornaram o mesmo resultado da correta)

---

### ✅ Resultado esperado:

```plaintext
--- Relatório de Baseline (Execução Local) ---
Total de Amostras Avaliadas: 50
Número de Sucessos (Execution Accuracy): 41
Número de Falhas: 9
Taxa de Sucesso Bruta: 82.00%

Resultados detalhados foram salvos em 'llama3_local_resultados_verificados.csv'
```

---

🔁 **Resumo**: Essa etapa valida, na prática, se as queries geradas funcionam corretamente, medindo a **Execution Accuracy**, uma métrica mais confiável que a simples comparação textual.



In [ ]:
try:
    df_resultados = pd.read_csv(ARQUIVO_DE_RESULTADOS)
except FileNotFoundError:
    print(f"ERRO: Arquivo '{ARQUIVO_DE_RESULTADOS}' não encontrado.")
    print("Certifique-se de que o script de avaliação local foi executado e gerou o arquivo CSV.")
    exit()

sucessos = 0
total = len(df_resultados)
resultados_detalhados = []

print(f"Iniciando a verificação por execução para {total} exemplos...")

for index, row in tqdm(df_resultados.iterrows(), total=total, desc="Verificando Resultados"):
    status = verificar_execucao(row['query_gerada'], row['query_correta'], row['db_path'])

    if status:
        sucessos += 1
        resultados_detalhados.append('Sucesso')
    else:
        resultados_detalhados.append('Falha')

# Adicionar uma coluna com os resultados da verificação para análise posterior
df_resultados['status_execucao'] = resultados_detalhados
df_resultados.to_csv("llama3_local_resultados_verificados.csv", index=False)

# --- Relatório Final de Baseline ---
print("\n--- Relatório de Baseline (Execução Local) ---")
print(f"Total de Amostras Avaliadas: {total}")
print(f"Número de Sucessos (Execution Accuracy): {sucessos}")
print(f"Número de Falhas: {total - sucessos}")

if total > 0:
    taxa_de_sucesso = (sucessos / total) * 100
    print(f"Taxa de Sucesso Bruta: {taxa_de_sucesso:.2f}%")

print("\nResultados detalhados foram salvos em 'llama3_local_resultados_verificados.csv'")

Iniciando a verificação por execução para 50 exemplos...


Verificando Resultados: 100%|██████████| 50/50 [00:00<00:00, 2554.17it/s]


--- Relatório de Baseline (Execução Local) ---
Total de Amostras Avaliadas: 50
Número de Sucessos (Execution Accuracy): 26
Número de Falhas: 24
Taxa de Sucesso Bruta: 52.00%

Resultados detalhados foram salvos em 'llama3_local_resultados_verificados.csv'


### 📌 Interpretação:
- **Execution Accuracy**: 52% das queries SQL geradas pelo modelo retornaram exatamente os mesmos resultados que a query correta (do dataset Spider).
- A métrica avalia se a **execução da query é semanticamente correta**, não apenas textual.
- O resultado é razoável para um modelo pequeno (3B) com LoRA, e pode ser melhorado com mais dados ou um modelo maior (ex: 7B ou 8B).

---

# Fase 2: Execução do Fine-Tuning

In [ ]:
import gc
from datasets import load_dataset
from transformers import (
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel, get_peft_model
from trl import SFTTrainer, SFTConfig

gc.collect()

# -- Configuração do LoRA
# Estes são os parâmetros do nosso "plugin" LoRA
lora_config = LoraConfig(
    r=16,  # Rank: A dimensão das matrizes do adaptador. Valores comuns: 8, 16, 32, 64.
    lora_alpha=32, # Um escalar de normalização. Convenção comum: alpha = 2 * r.
    target_modules=[ # Módulos do modelo onde os adaptadores serão injetados.
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
    ],
    lora_dropout=0.05, # Dropout para as camadas LoRA.
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
# -- Configuração do Treinamento
# Aqui definimos os hiperparâmetros do processo de treinamento.
training_arguments_v1 = SFTConfig(
    output_dir="./resultados/llama3-lora-exp1",
    num_train_epochs=1, # Número de vezes que o modelo verá o dataset inteiro
    per_device_train_batch_size=2, # Número de exemplos por lote por GPU
    gradient_accumulation_steps=2, # Acumula gradientes para simular um lote maior
    learning_rate=2e-4, # Taxa de aprendizado (Hiperparâmetro para experimentar)
    optim="paged_adamw_32bit", # Otimizador eficiente em memória
    logging_steps=25, # A cada quantos passos registrar o log de treino
    save_strategy="epoch", # Salvar o checkpoint ao final de cada época
    fp16=True, # Usar precisão de 16 bits para acelerar o treino
    push_to_hub=False, # Não salvar no Hugging Face
    report_to="none",  # <- ESSENCIAL para evitar WandB
)

average_tokens_across_devices is set to True but it is invalid when world size is1. Turn it to False automatically.


In [ ]:
# Função para formatar o prompt para o SFTTrainer
# O SFTTrainer espera uma única coluna de texto, formatada como uma conversa.
def formatar_exemplo_para_treino(exemplo):
    # Reutilizando a função de prompt anti-join que criamos
    schema = formatar_schema(exemplo)

    system_prompt = "Você é um especialista em SQL. Sua tarefa é gerar uma consulta SQL precisa a partir de uma pergunta em linguagem natural e o esquema do banco de dados fornecido. Não adicione nenhuma explicação, apenas o código SQL."

    # Formato Llama 3 original
    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>
### Esquema do Banco de Dados:
# {schema}
### Pergunta:
# {exemplo['question']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
{exemplo['query']}<|eot_id|>"""
    return {"text": prompt}


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# --- 3. CARREGAMENTO DO MODELO E TOKENIZADOR ---

gc.collect()
torch.cuda.empty_cache()

# Configuração de quantização para carregar o modelo em 4-bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    # bnb_4bit_quant_type="nf4",
    # bnb_4bit_compute_dtype=torch.bfloat16,
    # bnb_4bit_use_double_quant=True,
)

MODEL_ID = "meta-llama/Llama-3.2-3B-Instruct"
COMMIT_HASH = "0cb88a4f764b7a12671c53f0838cd831a0843b95"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    revision=COMMIT_HASH,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, COMMIT_HASH, token=hf_token)
tokenizer.pad_token = tokenizer.eos_token # Configurar o token de padding

model.config.use_cache = False # Necessário para o treino com LoRA

formatted_dataset = train_spider.map(formatar_exemplo_para_treino)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

In [ ]:
# --- 4. CONFIGURAÇÃO E EXECUÇÃO DO TREINADOR (SFTTrainer) ---
os.environ["WANDB_DISABLED"] = "true"

# SFTTrainer é uma classe otimizada para Supervised Fine-Tuning
trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset,
    peft_config=lora_config, # Passamos a configuração do LoRA aqui
    processing_class=tokenizer,
    args=training_arguments_v1, # Passamos os argumentos de treino aqui
)

print("Iniciando o fine-tuning com LoRA...")
trainer.train()

print("Fine-tuning concluído!")

# Salvar o adaptador LoRA treinado
# Note que isso salva apenas os pesos do adaptador (~dezenas de MB), não o modelo inteiro.
trainer.save_model(training_arguments_v1.output_dir)
print(f"Adaptador LoRA salvo em: {training_arguments_v1.output_dir}")

Adding EOS to train dataset:   0%|          | 0/7000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/7000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/7000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Iniciando o fine-tuning com LoRA...


/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:463: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss
25,1.225400
50,0.867200
75,0.728600
100,0.656300
125,0.580000
150,0.511800
175,0.520100
200,0.409100
225,0.401100
250,0.387000


Step,Training Loss
25,1.225400
50,0.867200
75,0.728600
100,0.656300
125,0.580000
150,0.511800
175,0.520100
200,0.409100
225,0.401100
250,0.387000


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Fine-tuning concluído!
Adaptador LoRA salvo em: ./resultados/llama3-lora-exp1


In [ ]:
# --- Execução da Avaliação ---
resultados = []
DB_DIR = "spider_data/database"

model = PeftModel.from_pretrained(model, training_arguments_v1.output_dir)
# `torch.no_grad()` desativa o cálculo de gradientes, economizando memória e acelerando a inferência.
with torch.no_grad():
    for i, exemplo in enumerate(tqdm(validation_spider.select(range(50)), desc="Avaliando Llama 3")):
        pergunta = exemplo['question']
        query_correta = exemplo['query']
        db_id = exemplo['db_id']

        schema_formatado = formatar_schema(exemplo)
        prompt_completo = criar_prompt_llama3(schema_formatado, pergunta)
        path_db = os.path.join(DB_DIR, db_id, f"{db_id}.sqlite")

        # 1. Tokenizar o prompt
        # O prompt é convertido em números que o modelo entende.
        inputs = tokenizer(prompt_completo, return_tensors="pt").to(model.device)

        # 2. Gerar a resposta com o modelo
        # O modelo gera a continuação do prompt tokenizado.
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            eos_token_id=tokenizer.eos_token_id,
            do_sample=False # Usamos do_sample=False para obter resultados mais consistentes
        )

        # 3. Decodificar a resposta
        # Os números gerados são convertidos de volta para texto.
        # `outputs[0]` pega o primeiro (e único) resultado do lote.
        # `inputs.input_ids.shape[1]:` fatia a saída para remover o prompt original e manter apenas o texto gerado.
        query_gerada = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True).strip()

        resultados.append({
            "db_id": exemplo['db_id'],
            "pergunta": pergunta,
            "query_correta": query_correta,
            "query_gerada": query_gerada,
            "db_path": path_db,
        })

        # Opcional: Limpar o cache da GPU em loops longos
        # torch.cuda.empty_cache()

df_resultados = pd.DataFrame(resultados)
df_resultados.to_csv("llama3_treinado_v1_baseline.csv", index=False)

print("\nAvaliação Local com Llama 3 concluída! Resultados salvos em 'llama3_treinado_v1_baseline.csv'")
# print(df_resultados.head())

/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
Avaliando Llama 3:   0%|          | 0/50 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Avaliando Llama 3:   2%|▏         | 1/50 [00:01<00:51,  1.04s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Avaliando Llama 3:   4%|▍         | 2/50 [00:02<00:49,  1.02s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for 


Avaliação Local com Llama 3 concluída! Resultados salvos em 'llama3_treinado_v1_baseline.csv'


In [ ]:
ARQUIVO_DE_RESULTADOS = "llama3_treinado_v1_baseline.csv"
try:
    df_resultados = pd.read_csv(ARQUIVO_DE_RESULTADOS)
except FileNotFoundError:
    print(f"ERRO: Arquivo '{ARQUIVO_DE_RESULTADOS}' não encontrado.")
    print("Certifique-se de que o script de avaliação local foi executado e gerou o arquivo CSV.")
    exit()

sucessos = 0
total = len(df_resultados)
resultados_detalhados = []

print(f"Iniciando a verificação por execução para {total} exemplos...")

for index, row in tqdm(df_resultados.iterrows(), total=total, desc="Verificando Resultados"):
    status = verificar_execucao(row['query_gerada'], row['query_correta'], row['db_path'])

    if status:
        sucessos += 1
        resultados_detalhados.append('Sucesso')
    else:
        resultados_detalhados.append('Falha')

# Adicionar uma coluna com os resultados da verificação para análise posterior
df_resultados['status_execucao'] = resultados_detalhados
df_resultados.to_csv("llama3_treinado_v1_resultados.csv", index=False)

# --- Relatório Final de Baseline ---
print("\n--- Relatório de Baseline (Execução Local) ---")
print(f"Total de Amostras Avaliadas: {total}")
print(f"Número de Sucessos (Execution Accuracy): {sucessos}")
print(f"Número de Falhas: {total - sucessos}")

if total > 0:
    taxa_de_sucesso = (sucessos / total) * 100
    print(f"Taxa de Sucesso Bruta: {taxa_de_sucesso:.2f}%")

print("\nResultados detalhados foram salvos em 'llama3_treinado_v1_resultados.csv'")

Iniciando a verificação por execução para 50 exemplos...


Verificando Resultados: 100%|██████████| 50/50 [00:00<00:00, 2487.67it/s]


--- Relatório de Baseline (Execução Local) ---
Total de Amostras Avaliadas: 50
Número de Sucessos (Execution Accuracy): 37
Número de Falhas: 13
Taxa de Sucesso Bruta: 74.00%

Resultados detalhados foram salvos em 'llama3_treinado_v1_resultados.csv'


## Treinando com outra configuração

In [ ]:
# Verifica se as variáveis existem antes de deletar
for var in ["trainer", "model", "tokenizer"]:
    if var in globals():
        del globals()[var]

gc.collect()
torch.cuda.empty_cache()


In [ ]:
# Clean up GPU memory from the previous training run
# Verifica se as variáveis existem antes de deletar
for var in ["trainer", "model", "tokenizer"]:
    if var in globals():
        del globals()[var]

gc.collect()
torch.cuda.empty_cache()


# Configuração de quantização para carregar o modelo em 4-bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    # bnb_4bit_quant_type="nf4",
    # bnb_4bit_compute_dtype=torch.bfloat16,
    # bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    revision=COMMIT_HASH,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, COMMIT_HASH, token=hf_token)
tokenizer.pad_token = tokenizer.eos_token

model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from transformers.trainer_callback import EarlyStoppingCallback

early_stopping = EarlyStoppingCallback(
    # monitor="eval_loss",  # Metric to monitor (e.g., 'eval_loss')
    # min_delta=0.01,                     # Minimum change to qualify as an improvement
    early_stopping_patience=3,            # Number of evaluations to wait before stopping
)

training_arguments_v2 = SFTConfig(
    output_dir="./resultados/llama3-lora-exp2",
    num_train_epochs=2, # Número de vezes que o modelo verá o dataset inteiro
    per_device_train_batch_size=2, # Número de exemplos por lote por GPU
    gradient_accumulation_steps=2, # Acumula gradientes para simular um lote maior
    learning_rate=2e-4, # Taxa de aprendizado (Hiperparâmetro para experimentar)
    optim="paged_adamw_32bit", # Otimizador eficiente em memória
    logging_steps=25, # A cada quantos passos registrar o log de treino
    save_strategy="epoch", # Salvar o checkpoint ao final de cada época
    fp16=True, # Usar precisão de 16 bits para acelerar o treino
    push_to_hub=False, # Não salvar no Hugging Face
)

trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset,
    peft_config=lora_config, # Passamos a configuração do LoRA aqui
    processing_class=tokenizer,
    args=training_arguments_v2, # Passamos os argumentos de treino aqui
    # callbacks=[early_stopping],
)


average_tokens_across_devices is set to True but it is invalid when world size is1. Turn it to False automatically.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
print("Iniciando o fine-tuning com LoRA...")
trainer.train()

print("Fine-tuning concluído!")

# Salvar o adaptador LoRA treinado
# Note que isso salva apenas os pesos do adaptador (~dezenas de MB), não o modelo inteiro.
trainer.save_model(training_arguments_v2.output_dir)
print(f"Adaptador LoRA salvo em: {training_arguments_v2.output_dir}")

Iniciando o fine-tuning com LoRA...


Step,Training Loss
25,1.227500
50,0.867200
75,0.728700
100,0.655600
125,0.581200
150,0.510800
175,0.513500
200,0.407000
225,0.401500
250,0.383500


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Fine-tuning concluído!
Adaptador LoRA salvo em: ./resultados/llama3-lora-exp2


In [ ]:
# --- Execução da Avaliação ---
resultados = []
DB_DIR = "spider_data/database"
model = PeftModel.from_pretrained(model, training_arguments_v2.output_dir)

# `torch.no_grad()` desativa o cálculo de gradientes, economizando memória e acelerando a inferência.
with torch.no_grad():
    for i, exemplo in enumerate(tqdm(validation_spider.select(range(50)), desc="Avaliando Llama 3")):
        pergunta = exemplo['question']
        query_correta = exemplo['query']
        db_id = exemplo['db_id']

        schema_formatado = formatar_schema(exemplo)
        prompt_completo = criar_prompt_llama3(schema_formatado, pergunta)
        path_db = os.path.join(DB_DIR, db_id, f"{db_id}.sqlite")

        # 1. Tokenizar o prompt
        # O prompt é convertido em números que o modelo entende.
        inputs = tokenizer(prompt_completo, return_tensors="pt").to(model.device)

        # 2. Gerar a resposta com o modelo
        # O modelo gera a continuação do prompt tokenizado.
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            eos_token_id=tokenizer.eos_token_id,
            do_sample=False # Usamos do_sample=False para obter resultados mais consistentes
        )

        # 3. Decodificar a resposta
        # Os números gerados são convertidos de volta para texto.
        # `outputs[0]` pega o primeiro (e único) resultado do lote.
        # `inputs.input_ids.shape[1]:` fatia a saída para remover o prompt original e manter apenas o texto gerado.
        query_gerada = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True).strip()

        resultados.append({
            "db_id": exemplo['db_id'],
            "pergunta": pergunta,
            "query_correta": query_correta,
            "query_gerada": query_gerada,
            "db_path": path_db,
        })

        # Opcional: Limpar o cache da GPU em loops longos
        # torch.cuda.empty_cache()

df_resultados = pd.DataFrame(resultados)
df_resultados.to_csv("llama3_treinado_v2_baseline.csv", index=False)

print("\nAvaliação Local com Llama 3 concluída! Resultados salvos em 'llama3_treinado_v2_baseline.csv'")
# print(df_resultados.head())

/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
Avaliando Llama 3:   0%|          | 0/50 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Avaliando Llama 3:   2%|▏         | 1/50 [00:01<00:49,  1.01s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Avaliando Llama 3:   4%|▍         | 2/50 [00:02<00:48,  1.01s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for 


Avaliação Local com Llama 3 concluída! Resultados salvos em 'llama3_treinado_v2_baseline.csv'


In [ ]:
ARQUIVO_DE_RESULTADOS = "llama3_treinado_v2_baseline.csv"
try:
    df_resultados = pd.read_csv(ARQUIVO_DE_RESULTADOS)
except FileNotFoundError:
    print(f"ERRO: Arquivo '{ARQUIVO_DE_RESULTADOS}' não encontrado.")
    print("Certifique-se de que o script de avaliação local foi executado e gerou o arquivo CSV.")
    exit()

sucessos = 0
total = len(df_resultados)
resultados_detalhados = []

print(f"Iniciando a verificação por execução para {total} exemplos...")

for index, row in tqdm(df_resultados.iterrows(), total=total, desc="Verificando Resultados"):
    status = verificar_execucao(row['query_gerada'], row['query_correta'], row['db_path'])

    if status:
        sucessos += 1
        resultados_detalhados.append('Sucesso')
    else:
        resultados_detalhados.append('Falha')

# Adicionar uma coluna com os resultados da verificação para análise posterior
df_resultados['status_execucao'] = resultados_detalhados
df_resultados.to_csv("llama3_treinado_v2_resultados.csv", index=False)

# --- Relatório Final de Baseline ---
print("\n--- Relatório de Baseline (Execução Local) ---")
print(f"Total de Amostras Avaliadas: {total}")
print(f"Número de Sucessos (Execution Accuracy): {sucessos}")
print(f"Número de Falhas: {total - sucessos}")

if total > 0:
    taxa_de_sucesso = (sucessos / total) * 100
    print(f"Taxa de Sucesso Bruta: {taxa_de_sucesso:.2f}%")

print("\nResultados detalhados foram salvos em 'llama3_treinado_v2_resultados.csv'")

Iniciando a verificação por execução para 50 exemplos...


Verificando Resultados: 100%|██████████| 50/50 [00:00<00:00, 2381.77it/s]


--- Relatório de Baseline (Execução Local) ---
Total de Amostras Avaliadas: 50
Número de Sucessos (Execution Accuracy): 38
Número de Falhas: 12
Taxa de Sucesso Bruta: 76.00%

Resultados detalhados foram salvos em 'llama3_treinado_v2_resultados.csv'


# Fase 3: Avaliação de Desempenho na Tarefa-Alvo com Métrica Customizada <br> Tá tudo errado


In [ ]:
# Importações necessárias no início do script
import pytest                    # Para rodar os testes automatizados
import traceback                 # Útil para debugar exceções
from typing import List, Any    # Para tipagem
from deepeval import assert_test            # Para validar test cases
from deepeval.metrics import BaseMetric     # Classe base para métricas customizadas
from deepeval.test_case import LLMTestCase  # Estrutura de cada exemplo de avaliação
import sqlite3                  # Para executar queries nos bancos do Spider


# =====================================================================================
# FASE 3.1: CÓDIGO DA MÉTRICA CUSTOMIZADA
# =====================================================================================

class ExecutionAccuracyMetric(BaseMetric):
    def __init__(self, threshold: float = 1.0):
        self.threshold = threshold

    def measure(self, test_case: LLMTestCase) -> float:
        if not test_case.actual_output or not test_case.expected_output:
            return 0.0
        if not test_case.context or not isinstance(test_case.context, list) or len(test_case.context) == 0:
            raise ValueError("O caminho para o banco de dados .sqlite deve ser fornecido em 'test_case.context'.")

        db_path = test_case.context[0]
        conn = None
        try:
            conn = sqlite3.connect(db_path)
            cursor = conn.cursor()

            try:
                cursor.execute(test_case.actual_output)
                actual_results = cursor.fetchall()
            except sqlite3.Error as e:
                self.reason = f"Falha na execução da query gerada: {e}"
                return 0.0

            try:
                cursor.execute(test_case.expected_output)
                expected_results = cursor.fetchall()
            except sqlite3.Error as e:
                self.reason = f"Falha na execução da query de referência: {e}"
                return 0.0

            if sorted(actual_results) == sorted(expected_results):
                self.score = 1.0
                self.reason = "Os resultados da execução foram idênticos."
            else:
                self.score = 0.0
                self.reason = f"Resultados divergentes. Esperado: {expected_results}, Obtido: {actual_results}"

            return self.score
        except Exception as e:
            self.reason = f"Erro inesperado: {e}\\n{traceback.format_exc()}"
            return 0.0
        finally:
            if conn:
                conn.close()

    def is_successful(self) -> bool:
        return self.score >= self.threshold

    @property
    def __name__(self):
        return "Execution Accuracy"



In [ ]:
test_code_string = r'''
import pytest
import json
import os
import sqlite3
import traceback
import torch
import re
from typing import List, Any
from deepeval import assert_test
from deepeval.metrics import BaseMetric
from deepeval.test_case import LLMTestCase
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# --- Definição da Métrica (CORRIGIDA para ser Assíncrona) ---
class ExecutionAccuracyMetric(BaseMetric):
    def __init__(self, threshold: float = 1.0):
        self.threshold = threshold
        # Define que esta métrica deve rodar de forma síncrona internamente
        # mesmo que seja chamada de um loop assíncrono.
        self._is_async = False

    def measure(self, test_case: LLMTestCase) -> float:
        """Este é o método síncrono que o DeepEval vai chamar."""
        if not test_case.actual_output or not test_case.expected_output:
            return 0.0
        if not test_case.context or not isinstance(test_case.context, list) or len(test_case.context) == 0:
            raise ValueError("O caminho para o banco de dados .sqlite deve ser fornecido em 'test_case.context'.")

        db_path = test_case.context[0]
        conn = None
        try:
            conn = sqlite3.connect(db_path)
            cursor = conn.cursor()

            # Limpa a query gerada pelo modelo
            cleaned_query = test_case.actual_output.strip()
            # Remove ```sql e ``` do início e fim
            if cleaned_query.startswith("```sql"):
                cleaned_query = cleaned_query[5:]
            if cleaned_query.endswith("```"):
                cleaned_query = cleaned_query[:-3]
            cleaned_query = cleaned_query.strip()

            try:
                cursor.execute(cleaned_query)
                actual_results = cursor.fetchall()
            except sqlite3.Error as e:
                self.reason = f"Falha na execução da query gerada (após limpeza): {e}. Query: '{cleaned_query}'"
                self.score = 0.0
                return self.score

            try:
                cursor.execute(test_case.expected_output)
                expected_results = cursor.fetchall()
            except sqlite3.Error as e:
                self.reason = f"Falha na execução da query de referência: {e}"
                self.score = 0.0
                return self.score

            if sorted(actual_results) == sorted(expected_results):
                self.score = 1.0
                self.reason = "Os resultados da execução foram idênticos."
            else:
                self.score = 0.0
                self.reason = f"Resultados divergentes. Esperado: {expected_results}, Obtido: {actual_results}"

            return self.score
        except Exception as e:
            self.reason = f"Erro inesperado: {e}\n{traceback.format_exc()}"
            self.score = 0.0
            return self.score
        finally:
            if conn:
                conn.close()

    def is_successful(self) -> bool:
        return self.score >= self.threshold

    @property
    def __name__(self):
        return "Execution Accuracy"

# --- Variáveis Globais e Funções Auxiliares (sem alterações) ---
model = None
tokenizer = None
schemas_map = None

def setup_model_and_data():
    global model, tokenizer, schemas_map
    if model is not None: return

    print("Carregando modelo, tokenizador e esquemas para o teste...")
    MODEL_ID = "meta-llama/Llama-3.2-3B-Instruct"
    COMMIT_HASH = "0cb88a4f764b7a12671c53f0838cd831a0843b95"

    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, revision=COMMIT_HASH)
    tokenizer.pad_token = tokenizer.eos_token

    bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        revision=COMMIT_HASH,
        quantization_config=bnb_config,
        device_map="auto",
    )

    with open('spider_data/tables.json', 'r') as f:
        schemas_data = json.load(f)
    schemas_map = {db['db_id']: db for db in schemas_data}
    print("Modelo e dados carregados.")

def criar_prompt_llama3(schema, question):
    system_prompt = "Você é um especialista em SQL. Sua tarefa é gerar uma consulta SQL precisa a partir de uma pergunta em linguagem natural e o esquema do banco de dados fornecido. Não adicione nenhuma explicação, apenas o código SQL."
    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n### Esquema do Banco de Dados:\n# {schema}\n### Pergunta:\n# {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""
    return prompt

def get_model_prediction(question: str, db_id: str) -> str:
    setup_model_and_data()

    schema_info = schemas_map[db_id]
    table_names = schema_info['table_names_original']
    column_names = schema_info['column_names_original']
    column_types = schema_info['column_types']
    schema_parts = []
    for table_idx, table_name in enumerate(table_names):
        cols = [f"{col[1]} {column_types[i]}" for i, col in enumerate(column_names) if col[0] == table_idx]
        schema_parts.append(f"CREATE TABLE {table_name} ({', '.join(cols)})")
    schema_formatado = "\n".join(schema_parts)

    prompt = criar_prompt_llama3(schema_formatado, question)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.eos_token_id)

    query_gerada = tokenizer.decode(outputs[0][inputs.input_ids.shape[-1]:], skip_special_tokens=True).strip()
    return query_gerada

def load_spider_dev_split():
    SPIDER_DEV_JSON = "spider_data/dev.json"
    SPIDER_DB_ROOT = "spider_data/database"
    with open(SPIDER_DEV_JSON, 'r') as f: data = json.load(f)
    samples_to_process = data[:5]
    test_cases = []
    for item in samples_to_process:
        db_id = item['db_id']
        db_path = os.path.join("/content", SPIDER_DB_ROOT, db_id, f"{db_id}.sqlite")
        if os.path.exists(db_path):
            test_cases.append(pytest.param(item['question'], item['query'], item['db_id'], db_path, id=f"{db_id}-{item['question'][:20]}"))
    return test_cases

# --- Definição do Teste ---
@pytest.mark.parametrize("question, ground_truth_sql, db_id, db_path", load_spider_dev_split())
def test_text_to_sql_execution(question: str, ground_truth_sql: str, db_id: str, db_path: str):
    model_generated_sql = get_model_prediction(question, db_id)

    test_case = LLMTestCase(
        input=question,
        actual_output=model_generated_sql,
        expected_output=ground_truth_sql,
        context=[db_path]
    )

    execution_metric = ExecutionAccuracyMetric(threshold=1.0)
    assert_test(test_case, [execution_metric])
'''

In [ ]:
# O nome do arquivo onde o teste será salvo
TEST_FILE_NAME = "test_colab.py"

# Escreve o código da string em um arquivo .py
with open(TEST_FILE_NAME, 'w') as f:
    f.write(test_code_string)

print(f"Arquivo de teste '{TEST_FILE_NAME}' criado com sucesso.")

# Agora, execute o pytest no arquivo que acabamos de criar.
!pytest -v --capture=no {TEST_FILE_NAME}

In [ ]:

import sqlite3
from deepeval.metrics import BaseMetric
from deepeval.test_case import LLMTestCase

class ExecutionAccuracy(BaseMetric):
    def __init__(self, name="ExecutionAccuracy"):
        self.name = name

    def measure(self, test_case: LLMTestCase) -> float:
        try:
            db_path = test_case.input  # o caminho do banco será passado aqui
            query_correta = test_case.expected_output
            query_gerada = test_case.actual_output

            # Se a query gerada for vazia ou inválida
            if not isinstance(query_gerada, str) or not query_gerada.strip():
                return 0.0

            conn = sqlite3.connect(f"file:{db_path}?mode=ro", uri=True)
            cursor = conn.cursor()

            # Executar ambas as queries
            cursor.execute(query_correta)
            resultado_correto = cursor.fetchall()

            cursor.execute(query_gerada)
            resultado_gerado = cursor.fetchall()

            # Converter resultados para string para garantir comparabilidade
            resultado_correto = sorted([tuple(map(str, row)) for row in resultado_correto])
            resultado_gerado = sorted([tuple(map(str, row)) for row in resultado_gerado])

            return 1.0 if resultado_correto == resultado_gerado else 0.0

        except Exception:
            return 0.0

        finally:
            if conn:
                conn.close()

In [ ]:
import pandas as pd
from deepeval.test_case import LLMTestCase

# Carrega os resultados do modelo fine-tuned
csv_path = "/content/llama3_treinado_v2_resultados.csv"
df = pd.read_csv(csv_path)

# Inicializa a métrica
metric = ExecutionAccuracy()

# Lista para armazenar os resultados
exec_scores = []

# Avaliação linha a linha
for _, row in df.iterrows():
    test_case = LLMTestCase(
        input=row["db_path"],               # Caminho para o banco
        actual_output=row["query_gerada"],  # Query gerada pelo modelo
        expected_output=row["query_correta"] # Query correta
    )
    score = metric.measure(test_case)
    exec_scores.append(score)

# Adiciona coluna com os resultados (1.0 ou 0.0)
df["exec_accuracy"] = exec_scores

# Calcula taxa de sucesso
taxa_sucesso = sum(exec_scores) / len(exec_scores) * 100

# Salva novo CSV com a coluna de métricas
output_path = "/content/llama3_treinado_v2_resultados_exec_accuracy.csv"
df.to_csv(output_path, index=False)

exec_scores.append(taxa_sucesso)
print(f"Taxa de Sucesso: {taxa_sucesso:.2f}%")



Taxa de Sucesso: 76.00%


In [ ]:
import pandas as pd
from deepeval.test_case import LLMTestCase

# Carrega os resultados do modelo fine-tuned
csv_path = "/content/llama3_treinado_v1_resultados.csv"
df = pd.read_csv(csv_path)

# Inicializa a métrica
metric = ExecutionAccuracy()

# Lista para armazenar os resultados
exec_scores = []

# Avaliação linha a linha
for _, row in df.iterrows():
    test_case = LLMTestCase(
        input=row["db_path"],               # Caminho para o banco
        actual_output=row["query_gerada"],  # Query gerada pelo modelo
        expected_output=row["query_correta"] # Query correta
    )
    score = metric.measure(test_case)
    exec_scores.append(score)

# Adiciona coluna com os resultados (1.0 ou 0.0)
df["exec_accuracy"] = exec_scores

# Calcula taxa de sucesso
taxa_sucesso = sum(exec_scores) / len(exec_scores) * 100

# Salva novo CSV com a coluna de métricas
output_path = "/content/llama3_treinado_v1_resultados_exec_accuracy.csv"
df.to_csv(output_path, index=False)

exec_scores.append(taxa_sucesso)
print(f"Taxa de Sucesso: {taxa_sucesso:.2f}%")



Taxa de Sucesso: 74.00%


# Vamos implementar Automating Text-To-SQL Execution Accuracy Measurements: QueryCraft
*link de referencia: https://medium.com/@Shivam_Solanki/querycraft-automating-nl2sql-execution-accuracy-measurements-5d7ab8398232*


https://colab.research.google.com/github/milvus-io/bootcamp/blob/master/integration/evaluation_with_deepeval.ipynb#scrollTo=6riSNR_Z0L-T




In [ ]:
import sqlite3
import re
from deepeval.metrics import BaseMetric
from deepeval.test_case import LLMTestCase

class QueryCraftExecutionAccuracy(BaseMetric):
    def __init__(self, name="QueryCraftExecutionAccuracy"):
        self.name = name

    def _sanitize_query(self, query: str) -> str:
        if not query:
            return ""

        query = query.strip()

        # Remove alias (Ex: AS T1)
        query = re.sub(r"AS\s+\w+", "", query, flags=re.IGNORECASE)

        # Remove múltiplos espaços
        query = re.sub(r"\s+", " ", query)

        # Remove espaços ao redor de símbolos
        query = re.sub(r"\s*([=,()])\s*", r"\1", query)

        return query.strip()

    def measure(self, test_case: LLMTestCase) -> float:
        conn = None
        try:
            db_path = test_case.input
            query_correta = self._sanitize_query(test_case.expected_output)
            query_gerada = self._sanitize_query(test_case.actual_output)

            if not query_gerada:
                return 0.0

            conn = sqlite3.connect(f"file:{db_path}?mode=ro", uri=True)
            cursor = conn.cursor()

            try:
                cursor.execute(query_gerada)
                resultado_gerado = cursor.fetchall()
            except sqlite3.Error:
                return 0.0

            try:
                cursor.execute(query_correta)
                resultado_correto = cursor.fetchall()
            except sqlite3.Error:
                return 0.0

            resultado_gerado = sorted([tuple(map(str, row)) for row in resultado_gerado])
            resultado_correto = sorted([tuple(map(str, row)) for row in resultado_correto])

            return 1.0 if resultado_gerado == resultado_correto else 0.0

        except Exception:
            return 0.0
        finally:
            if conn:
                conn.close()


In [ ]:
import pytest
import pandas as pd
from deepeval import assert_test
from deepeval.test_case import LLMTestCase

# Carregar CSV gerado com as queries
CSV_PATH = "llama3_treinado_v1_resultados.csv"
df = pd.read_csv(CSV_PATH)

# Gerar os test cases a partir do CSV
test_cases = []
for _, row in df.iterrows():
    test_case = LLMTestCase(
        input=row["db_path"],
        actual_output=row["query_gerada"],
        expected_output=row["query_correta"]
    )
    test_cases.append(test_case)

# Criar a métrica
metric = QueryCraftExecutionAccuracy()

@pytest.mark.parametrize("test_case", test_cases)
def test_query_execution_accuracy(test_case):
    assert_test(test_case, [metric])


In [ ]:
import pandas as pd
from deepeval.test_case import LLMTestCase
from tqdm import tqdm

# Carregar CSV com as queries
csv_path = "/content/llama3_treinado_v1_resultados.csv"
df = pd.read_csv(csv_path)

# Inicializar a métrica
metric = QueryCraftExecutionAccuracy()

# Avaliar cada exemplo manualmente
scores = []
for _, row in tqdm(df.iterrows(), total=len(df), desc="Executando Avaliação Local"):
    test_case = LLMTestCase(
        input=row["db_path"],
        actual_output=row["query_gerada"],
        expected_output=row["query_correta"]
    )
    score = metric.measure(test_case)
    scores.append(score)

# Adicionar resultado no DataFrame
df["querycraft_execution_accuracy"] = scores

# Calcular taxa de sucesso
taxa_sucesso = (sum(scores) / len(scores)) * 100
print(f"Taxa de Sucesso com QueryCraft: {taxa_sucesso:.2f}%")

# Salvar resultados
df.to_csv("llama3_treinado_v1_resultados_querycraft.csv", index=False)
print("Resultados com QueryCraft salvos!")


Executando Avaliação Local: 100%|██████████| 50/50 [00:00<00:00, 2225.31it/s]

Taxa de Sucesso com QueryCraft: 50.00%
Resultados com QueryCraft salvos!


In [ ]:
def handle_sql_error(error_message):
    if "no such column" in error_message:
        return "Coluna ausente"
    if "syntax error" in error_message:
        return "Erro de sintaxe"
    if "ambiguous column name" in error_message:
        return "Nome de coluna ambíguo"
    return "Outro erro"
import re

def reformat_query(query):
    if not query:
        return ""
    query = query.strip()
    query = re.sub(r"AS\s+\w+", "", query, flags=re.IGNORECASE)  # Remove aliases
    query = re.sub(r"\s+", " ", query)
    query = re.sub(r"\s*\*\s*", "*", query)
    return query.strip()
def is_valid_sql(cursor, query):
    try:
        cursor.execute(query)
        return True
    except sqlite3.Error:
        return False
def multiset_eq(list1, list2):
    return sorted([tuple(map(str, row)) for row in list1]) == sorted([tuple(map(str, row)) for row in list2])
def evaluate_query_execution(db_path, gold_query, predicted_query):
    conn = sqlite3.connect(f"file:{db_path}?mode=ro", uri=True)
    cursor = conn.cursor()

    resultado_correto = []
    resultado_gerado = []
    erro_sql = None

    try:
        cursor.execute(gold_query)
        resultado_correto = cursor.fetchall()
    except sqlite3.Error as e:
        conn.close()
        return {
            "valid_gold": False,
            "valid_pred": False,
            "precisao_inferencia": 0.0,
            "precisao_pos_processamento": 0.0,
            "erro": handle_sql_error(str(e))
        }

    try:
        cursor.execute(predicted_query)
        resultado_gerado = cursor.fetchall()
        valid_pred = True
    except sqlite3.Error as e:
        valid_pred = False
        erro_sql = handle_sql_error(str(e))

    # Precisão antes da correção
    precisao_inferencia = 1.0 if valid_pred and multiset_eq(resultado_correto, resultado_gerado) else 0.0

    # --- Pós-processamento ---
    predicted_query_ref = reformat_query(predicted_query)

    try:
        cursor.execute(predicted_query_ref)
        resultado_gerado_ref = cursor.fetchall()
        valid_pred_ref = True
    except sqlite3.Error:
        valid_pred_ref = False

    precisao_pos_processamento = 1.0 if valid_pred_ref and multiset_eq(resultado_correto, resultado_gerado_ref) else 0.0

    conn.close()

    return {
        "valid_gold": True,
        "valid_pred": valid_pred,
        "precisao_inferencia": precisao_inferencia,
        "precisao_pos_processamento": precisao_pos_processamento,
        "erro": erro_sql
    }
import pandas as pd
from tqdm import tqdm

def avaliar_todos(df):
    resultados = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Avaliando com QueryCraft"):
        resultado = evaluate_query_execution(
            db_path=row["db_path"],
            gold_query=row["query_correta"],
            predicted_query=row["query_gerada"]
        )
        resultados.append({
            "db_id": row["db_id"],
            "pergunta": row["pergunta"],
            "query_correta": row["query_correta"],
            "query_gerada": row["query_gerada"],
            "erro": resultado["erro"],
            "precisao_inferencia": resultado["precisao_inferencia"],
            "precisao_pos_processamento": resultado["precisao_pos_processamento"]
        })
    return pd.DataFrame(resultados)
# Carregar CSV
df_input = pd.read_csv("/content/llama3_treinado_v1_resultados.csv")

# Avaliar
df_resultados_querycraft = avaliar_todos(df_input)

# Salvar
df_resultados_querycraft.to_csv("llama3_treinado_v1_resultados_querycraft.csv", index=False)
print("Resultados salvos com sucesso!")


Avaliando com QueryCraft: 100%|██████████| 50/50 [00:00<00:00, 1413.33it/s]

Resultados salvos com sucesso!


In [ ]:
# ========================== IMPORTAÇÕES ==========================
import sqlite3
import pandas as pd
from deepeval.metrics import BaseMetric
from deepeval.test_case import LLMTestCase

# ========================== FUNÇÕES AUXILIARES ==========================

def is_valid_sql(query, db_path):
    """Verifica se a query é executável."""
    try:
        conn = sqlite3.connect(f"file:{db_path}?mode=ro", uri=True)
        cursor = conn.cursor()
        cursor.execute(query)
        conn.close()
        return True
    except Exception:
        return False

def preprocess_query(query):
    """Aplica correções simples no formato da query."""
    if query is None or not isinstance(query, str):
        return ""

    # Correção de alias e sintaxe comum
    query = query.replace("t1.*", "*")
    query = query.replace("T1.*", "*")
    query = query.replace(";", "")  # Remove ponto e vírgula extra
    query = query.strip()

    return query

def execute_query(query, db_path):
    """Executa uma query no banco sqlite e retorna os resultados ordenados."""
    try:
        conn = sqlite3.connect(f"file:{db_path}?mode=ro", uri=True)
        cursor = conn.cursor()
        cursor.execute(query)
        results = cursor.fetchall()
        results = sorted([tuple(map(str, row)) for row in results])
        conn.close()
        return results
    except Exception:
        return None

def compare_results(result1, result2):
    """Compara resultados SQL desconsiderando a ordem das linhas."""
    if result1 is None or result2 is None:
        return False
    return result1 == result2

# ========================== MÉTRICA CUSTOMIZADA ==========================

class ExecutionAccuracy(BaseMetric):
    def __init__(self, name="ExecutionAccuracy"):
        self.name = name

    def measure(self, test_case: LLMTestCase) -> float:
        db_path = test_case.input
        query_correta = test_case.expected_output
        query_gerada = test_case.actual_output

        if not query_gerada or not isinstance(query_gerada, str) or not query_gerada.strip():
            return 0.0

        query_gerada_corrigida = preprocess_query(query_gerada)

        if not is_valid_sql(query_gerada_corrigida, db_path):
            return 0.0

        resultado_correto = execute_query(query_correta, db_path)
        resultado_gerado = execute_query(query_gerada_corrigida, db_path)

        if compare_results(resultado_correto, resultado_gerado):
            return 1.0
        return 0.0

# ========================== EXECUÇÃO DO PIPELINE ==========================

# Carregar os dados
df_input = pd.read_csv("/content/llama3_treinado_v1_resultados.csv")

# Inicializar a métrica
metric = ExecutionAccuracy()

# Avaliar cada linha
exec_scores = []
for idx, row in df_input.iterrows():
    test_case = LLMTestCase(
        input=row["db_path"],
        actual_output=row["query_gerada"],
        expected_output=row["query_correta"]
    )
    score = metric.measure(test_case)
    exec_scores.append(score)

# Salvar os resultados
df_input["execution_accuracy_corrected"] = exec_scores
df_input.to_csv("llama3_treinado_v1_resultados_exec_accuracy_corrected.csv", index=False)

# Calcular taxa de sucesso
taxa_sucesso = (sum(exec_scores) / len(exec_scores)) * 100
print(f"Taxa de Sucesso Após Pós-Processamento: {taxa_sucesso:.2f}%")
print("Resultados salvos em: llama3_treinado_v1_resultados_exec_accuracy_corrected.csv")


Taxa de Sucesso Após Pós-Processamento: 74.00%
Resultados salvos em: llama3_treinado_v1_resultados_exec_accuracy_corrected.csv


In [ ]:
# ============================
# IMPORTAÇÕES
# ============================
import pandas as pd
import sqlite3
from tqdm import tqdm
from deepeval.metrics import BaseMetric
from deepeval.test_case import LLMTestCase

# ============================
# MÉTRICA CUSTOMIZADA
# ============================
class ExecutionAccuracy(BaseMetric):
    def __init__(self, name="ExecutionAccuracy"):
        self.name = name

    def measure(self, test_case: LLMTestCase) -> float:
        try:
            db_path = test_case.input
            query_correta = test_case.expected_output
            query_gerada = test_case.actual_output

            if not isinstance(query_gerada, str) or not query_gerada.strip():
                return 0.0

            conn = sqlite3.connect(f"file:{db_path}?mode=ro", uri=True)
            cursor = conn.cursor()

            cursor.execute(query_correta)
            resultado_correto = cursor.fetchall()

            cursor.execute(query_gerada)
            resultado_gerado = cursor.fetchall()

            resultado_correto = sorted([tuple(map(str, row)) for row in resultado_correto])
            resultado_gerado = sorted([tuple(map(str, row)) for row in resultado_gerado])

            return 1.0 if resultado_correto == resultado_gerado else 0.0

        except Exception:
            return 0.0

        finally:
            if conn:
                conn.close()

# ============================
# AVALIAÇÃO MANUAL COM DEEPEVAL
# ============================
print("Carregando CSV...")
df = pd.read_csv("/content/llama3_treinado_v1_resultados.csv")

test_cases = []
for _, row in tqdm(df.iterrows(), total=len(df), desc="Preparando os Test Cases"):
    test_case = LLMTestCase(
        input=row["db_path"],
        actual_output=row["query_gerada"],
        expected_output=row["query_correta"]
    )
    test_cases.append(test_case)

execution_metric = ExecutionAccuracy()
resultados_detalhados = []

acertos = 0

print("\nIniciando a avaliação manual...")
for test_case in tqdm(test_cases, desc="Avaliando Test Cases"):
    score = execution_metric.measure(test_case)

    if score == 1.0:
        acertos += 1
        status = "Sucesso"
    else:
        status = "Falha"

    resultados_detalhados.append({
        "db_path": test_case.input,
        "query_correta": test_case.expected_output,
        "query_gerada": test_case.actual_output,
        "status_execucao": status
    })

# Criar DataFrame com os resultados
df_resultados = pd.DataFrame(resultados_detalhados)
df_resultados.to_csv("/content/resultado_deepeval_manual.csv", index=False)

total = len(test_cases)
taxa_acerto = (acertos / total) * 100

print(f"\n--- Avaliação concluída ---")
print(f"Total de Test Cases: {total}")
print(f"Total de Acertos: {acertos}")
print(f"Taxa de Acerto: {taxa_acerto:.2f}%")
print("Resultados salvos em 'resultado_deepeval_manual.csv'")


Carregando CSV...


Preparando os Test Cases: 100%|██████████| 50/50 [00:00<00:00, 17835.96it/s]



Iniciando a avaliação manual...


Avaliando Test Cases: 100%|██████████| 50/50 [00:00<00:00, 3452.84it/s]


--- Avaliação concluída ---
Total de Test Cases: 50
Total de Acertos: 37
Taxa de Acerto: 74.00%
Resultados salvos em 'resultado_deepeval_manual.csv'


In [ ]:
!pip install deepeval==0.21.78



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.9/301.9 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 118.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45

In [ ]:
# ============================
# IMPORTAÇÕES
# ============================
import pandas as pd
import sqlite3
from tqdm import tqdm
from deepeval.metrics import BaseMetric
from deepeval.test_case import LLMTestCase
from deepeval import evaluate

# ============================
# MÉTRICA CUSTOMIZADA ATUALIZADA
# ============================
class ExecutionAccuracy(BaseMetric):
    def __init__(self, name="ExecutionAccuracy"):
        self.name = name
        self.is_async = False  # Deve estar definido assim para rodar sincronamente

    def measure(self, test_case: LLMTestCase) -> float:
        try:
            db_path = test_case.input
            query_correta = test_case.expected_output
            query_gerada = test_case.actual_output

            if not isinstance(query_gerada, str) or not query_gerada.strip():
                return 0.0

            conn = sqlite3.connect(f"file:{db_path}?mode=ro", uri=True)
            cursor = conn.cursor()

            cursor.execute(query_correta)
            resultado_correto = cursor.fetchall()

            cursor.execute(query_gerada)
            resultado_gerado = cursor.fetchall()

            resultado_correto = sorted([tuple(map(str, row)) for row in resultado_correto])
            resultado_gerado = sorted([tuple(map(str, row)) for row in resultado_gerado])

            return 1.0 if resultado_correto == resultado_gerado else 0.0

        except Exception:
            return 0.0

        finally:
            if conn:
                conn.close()

# ============================
# CARREGAMENTO DOS DADOS
# ============================
print("Carregando CSV...")
df = pd.read_csv("/content/llama3_treinado_v1_resultados.csv")

test_cases = []
for _, row in tqdm(df.iterrows(), total=len(df), desc="Preparando os Test Cases"):
    test_case = LLMTestCase(
        input=row["db_path"],
        actual_output=row["query_gerada"],
        expected_output=row["query_correta"]
    )
    test_cases.append(test_case)

# ============================
# AVALIAÇÃO COM DEEPEVAL ATUALIZADO
# ============================
print("\nIniciando a avaliação com DeepEval...")

execution_metric = ExecutionAccuracy()

# Chamando o evaluate moderno
result = evaluate(
    test_cases=test_cases,
    metrics=[execution_metric],
    print_results=True  # Ativa a exibição automática no terminal
)

print("\n✅ Avaliação concluída com sucesso!")


Carregando CSV...


Preparando os Test Cases: 100%|██████████| 50/50 [00:00<00:00, 18717.89it/s]


Iniciando a avaliação com DeepEval...


TypeError: evaluate() got an unexpected keyword argument 'print_results'

# Fase 4: Análise Quantitativa de Regressão de Capacidade


> Objetivo: Avaliar a capacidade do modelo em múltiplos domínios após o fine-tuning.

In [ ]:
# Exibir um exemplo completo do dataset
evaluation_suite['evaluation'][0]


{'question': 'Taurek claims that if faced with the choice between saving the life of X or the arm of Y:',
 'subject': 'philosophy',
 'choices': ['Y is obligated to choose saving the life of X.',
  'we are obligated to choose saving the life of X.',
  'both a and b.',
  'neither a nor b.'],
 'answer': 3}

In [ ]:
# Vamos selecionar as 4 primeiras questões como exemplos 4-shot
exemplos_4shot = []

for i in range(4):
    exemplo = evaluation_suite['evaluation'][i]
    exemplos_4shot.append({
        "question": exemplo['question'],
        "choices": exemplo['choices'],
        "answer_index": exemplo['answer']  # Vamos manter como número
    })


In [ ]:
def criar_prompt_mmlu_4shot(exemplos, pergunta_atual):
    prompt = ""

    # Adicionar os 4 exemplos no prompt
    for exemplo in exemplos:
        prompt += f"Exemplo:\nPergunta: {exemplo['question']}\n"
        prompt += f"0.) {exemplo['choices'][0]}\n1.) {exemplo['choices'][1]}\n"
        prompt += f"2.) {exemplo['choices'][2]}\n3.) {exemplo['choices'][3]}\n"
        prompt += f"Resposta: {exemplo['answer_index']}\n\n"

    # Adicionar a pergunta que será avaliada
    prompt += f"Pergunta:\n{pergunta_atual['question']}\n"
    prompt += f"0.) {pergunta_atual['choices'][0]}\n1.) {pergunta_atual['choices'][1]}\n"
    prompt += f"2.) {pergunta_atual['choices'][2]}\n3.) {pergunta_atual['choices'][3]}\n"
    prompt += "Responda apenas com o número da alternativa correta (0, 1, 2 ou 3). Não adicione explicações.\n"
    prompt += "Resposta:"

    return prompt


> Testando com o modelo base

In [ ]:
# Clean up GPU memory from the previous training run
# Verifica se as variáveis existem antes de deletar
for var in ["trainer", "model", "tokenizer"]:
    if var in globals():
        del globals()[var]

gc.collect()
torch.cuda.empty_cache()

In [ ]:
# --- Configuração ---
MODEL_ID = "meta-llama/Llama-3.2-3B-Instruct"

COMMIT_HASH = "0cb88a4f764b7a12671c53f0838cd831a0843b95"

print("Carregando o tokenizador...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, COMMIT_HASH, token=hf_token)

# Configuração de quantização para carregar o modelo em 4-bit
bnb_config = BitsAndBytesConfig(
    # load_in_8bit=True,
    load_in_4bit=True,
    # bnb_4bit_quant_type="nf4",
    # bnb_4bit_compute_dtype=torch.bfloat16,
    # bnb_4bit_use_double_quant=True,
)

# Carregar o modelo base com quantização
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    revision=COMMIT_HASH,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

Carregando o tokenizador...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from tqdm import tqdm
import pandas as pd

resultados_base = []
model.eval()

with torch.no_grad():
    for i in tqdm(range(4, len(evaluation_suite['evaluation'])), desc="Avaliando modelo base"):
        pergunta_atual = evaluation_suite['evaluation'][i]

        # Criar o prompt 4-shot
        prompt = criar_prompt_mmlu_4shot(exemplos_4shot, pergunta_atual)

        # Tokenizar o prompt
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        # Gerar a resposta
        outputs = model.generate(
            **inputs,
            max_new_tokens=10,
            eos_token_id=tokenizer.eos_token_id,
            do_sample=False
        )

        # Decodificar a resposta
        resposta_modelo = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True).strip()
        print(f"Resposta bruta do modelo: {resposta_modelo}")


        # Tentar extrair a primeira opção numérica (0, 1, 2, 3) da resposta
        resposta_num = None
        for char in resposta_modelo:
            if char in ['0', '1', '2', '3']:
                resposta_num = int(char)
                break

        # Se não encontrar nenhuma opção, considera como erro
        if resposta_num is None:
            resposta_num = -1  # Marca como resposta inválida

        resposta_correta = pergunta_atual['answer']

        resultados_base.append({
            "question": pergunta_atual['question'],
            "resposta_modelo": resposta_num,
            "resposta_correta": resposta_correta,
            "subject": pergunta_atual['subject']
        })

# Converter para DataFrame
df_base = pd.DataFrame(resultados_base)
df_base.to_csv("/content/resultados_base_mmlu.csv", index=False)
print("Resultados no modelo base salvos!")


Avaliando modelo base:   0%|          | 0/146 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Avaliando modelo base:   1%|          | 1/146 [00:00<00:29,  4.86it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Avaliando modelo base:   1%|▏         | 2/146 [00:00<00:28,  4.97it/s]

Resposta bruta do modelo: 2
Resposta bruta do modelo: 2


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Avaliando modelo base:   2%|▏         | 3/146 [00:00<00:28,  4.96it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Resposta bruta do modelo: 3


Avaliando modelo base:   3%|▎         | 4/146 [00:00<00:29,  4.81it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Avaliando modelo base:   3%|▎         | 5/146 [00:01<00:28,  4.90it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

Resposta bruta do modelo: 1


Avaliando modelo base:  99%|█████████▊| 144/146 [00:29<00:00,  4.85it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Avaliando modelo base:  99%|█████████▉| 145/146 [00:30<00:00,  4.91it/s]

Resposta bruta do modelo: 3
Resposta bruta do modelo: 3


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Avaliando modelo base: 100%|██████████| 146/146 [00:30<00:00,  4.83it/s]

Resposta bruta do modelo: 3
Resultados no modelo base salvos!


> agora vamos para o modelo finetunado

In [ ]:
from peft import PeftModel

# Carregar o modelo base novamente
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    revision=COMMIT_HASH,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

# Carregar o adaptador LoRA treinado
model = PeftModel.from_pretrained(model, "/content/resultados/llama3-lora-exp2")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from tqdm import tqdm
import pandas as pd

resultados_base = []
model.eval()

with torch.no_grad():
    for i in tqdm(range(4, len(evaluation_suite['evaluation'])), desc="Avaliando modelo finetunado"):
        pergunta_atual = evaluation_suite['evaluation'][i]

        # Criar o prompt 4-shot
        prompt = criar_prompt_mmlu_4shot(exemplos_4shot, pergunta_atual)

        # Tokenizar o prompt
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        # Gerar a resposta
        outputs = model.generate(
            **inputs,
            max_new_tokens=10,
            eos_token_id=tokenizer.eos_token_id,
            do_sample=False
        )

        # Decodificar a resposta
        resposta_modelo = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True).strip()
        print(f"Resposta bruta do modelo: {resposta_modelo}")


        # Tentar extrair a primeira opção numérica (0, 1, 2, 3) da resposta
        resposta_num = None
        for char in resposta_modelo:
            if char in ['0', '1', '2', '3']:
                resposta_num = int(char)
                break

        # Se não encontrar nenhuma opção, considera como erro
        if resposta_num is None:
            resposta_num = -1  # Marca como resposta inválida

        resposta_correta = pergunta_atual['answer']

        resultados_base.append({
            "question": pergunta_atual['question'],
            "resposta_modelo": resposta_num,
            "resposta_correta": resposta_correta,
            "subject": pergunta_atual['subject']
        })

# Converter para DataFrame
df_base = pd.DataFrame(resultados_base)
df_base.to_csv("mmlu_results_finetuned.csv", index=False)
print("Resultados no modelo Finetunado salvos!")


Avaliando modelo finetunado:   0%|          | 0/146 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Avaliando modelo finetunado:   1%|          | 1/146 [00:00<02:11,  1.10it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Resposta bruta do modelo: 3

Pergunta:
A time series


Avaliando modelo finetunado:   1%|▏         | 2/146 [00:01<02:10,  1.10it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Resposta bruta do modelo: 2
Exemplo:
Pergunta:


Avaliando modelo finetunado:   2%|▏         | 3/146 [00:02<02:06,  1.13it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

Resposta bruta do modelo: 3
Exemplo:
Pergunta:


Avaliando modelo finetunado:  99%|█████████▉| 145/146 [01:57<00:00,  1.17it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Resposta bruta do modelo: 3

Pergunta:
The main argument


Avaliando modelo finetunado: 100%|██████████| 146/146 [01:58<00:00,  1.24it/s]

Resposta bruta do modelo: 3
Exemplo:
Pergunta:
Resultados no modelo Finetunado salvos!


In [ ]:
# Recarregar os arquivos após reset
import pandas as pd

# Carregar os resultados dos dois modelos
baseline_results = pd.read_csv("/content/resultados_base_mmlu.csv")
finetuned_results = pd.read_csv("/content/mmlu_results_finetuned.csv")

# Função para calcular acurácia geral e por categoria
def calcular_acuracia(df):
    # Marca 1 se a resposta do modelo foi igual à resposta correta, 0 caso contrário
    df['correto'] = (df['resposta_modelo'] == df['resposta_correta']).astype(int)

    # Acurácia geral
    acuracia_geral = df['correto'].mean() * 100

    # Acurácia por categoria
    acuracia_por_categoria = df.groupby('subject')['correto'].mean() * 100

    return acuracia_geral, acuracia_por_categoria

# Calcular acurácias
acuracia_base, acuracia_base_por_categoria = calcular_acuracia(baseline_results)
acuracia_finetuned, acuracia_finetuned_por_categoria = calcular_acuracia(finetuned_results)

# Calcular variação percentual total
variacao_percentual_total = ((acuracia_finetuned - acuracia_base) / acuracia_base) * 100

# Calcular variação percentual por categoria
variacao_por_categoria = ((acuracia_finetuned_por_categoria - acuracia_base_por_categoria) / acuracia_base_por_categoria) * 100

# Organizar os resultados
relatorio = pd.DataFrame({
    'Acurácia Base (%)': acuracia_base_por_categoria,
    'Acurácia Fine-Tuned (%)': acuracia_finetuned_por_categoria,
    'Variação Percentual (%)': variacao_por_categoria
}).reset_index()

# Adicionar a linha de resumo geral
linha_resumo = pd.DataFrame({
    'subject': ['GERAL'],
    'Acurácia Base (%)': [acuracia_base],
    'Acurácia Fine-Tuned (%)': [acuracia_finetuned],
    'Variação Percentual (%)': [variacao_percentual_total]
})

relatorio_final = pd.concat([linha_resumo, relatorio], ignore_index=True)
print(relatorio_final)

                    subject  Acurácia Base (%)  Acurácia Fine-Tuned (%)  \
0                     GERAL          39.726027                39.041096   
1  college_computer_science          36.734694                42.857143   
2              econometrics          32.653061                28.571429   
3                philosophy          50.000000                45.833333   

   Variação Percentual (%)  
0                -1.724138  
1                16.666667  
2               -12.500000  
3                -8.333333  


O modelo ganhou desempenho na categoria de STEM (Computer Science), o que é esperado, pois foi fine-tunado para Text-to-SQL.

O modelo perdeu capacidade geral em Humanities e Social Sciences, indicando um provável fenômeno de regressão de capacidade.

A acurácia geral se manteve quase estável (-1,72%), o que pode ser visto como um bom trade-off.

Houve perda de desempenho nas áreas de Ciências Sociais (Econometrics) e Humanidades (Philosophy), indicando regressão de capacidade nestes domínios.

No resultado geral, a variação foi pequena, com uma leve queda de 1,72% na acurácia total, sugerindo que o modelo manteve boa parte de sua capacidade geral.